In [1]:
%load_ext line_profiler

from pathlib import Path
import sys

import numpy as np

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[0])
print(parent_dir)
# Add to sys.path

path_set = set(sys.path)
if parent_dir not in path_set:
    sys.path.insert(0, parent_dir)

print(sys.path)

/Users/colinbull/appdev/dfe/sfb/education-benchmarking-and-insights/data-pipeline
['/Users/colinbull/appdev/dfe/sfb/education-benchmarking-and-insights/data-pipeline', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/colinbull/Library/Caches/pypoetry/virtualenvs/fbit-data-pipeline-aJYNke-B-py3.12/lib/python3.12/site-packages']


# Metric RAG

* median, decile, status (Red / Amber / Green), percentage
* All cost catgeories - cost per pupil
* Utilities and Premises and staff - per m^2

In [2]:
import pandas as pd
import src.pipeline.rag as rag
import src.pipeline.config as config
import glob 
import os
import time

# Create and clean directory
from pathlib import Path
Path("output/metric-rag").mkdir(parents=True, exist_ok=True)

files = glob.glob("output/metric-rag/*")
for f in files:
    os.remove(f)

## Loading saved comparator sets

In [3]:
raw_data = pd.read_parquet('output/comparator-sets/maintained_schools.parquet')
raw_comparators = pd.read_parquet('output/comparator-sets/ms_comparators.parquet')

In [4]:
raw_comparators.head(10)

,OfstedRating (name),Percentage SEN,Percentage Free school meals,Number of pupils,Total Internal Floor Area,Age Average Score,GOR (name),SchoolPhaseType,Percentage Primary Need SPLD,Percentage Primary Need MLD,Percentage Primary Need PMLD,Percentage Primary Need SEMH,Percentage Primary Need SLCN,Percentage Primary Need HI,Percentage Primary Need MSI,Percentage Primary Need PD,Percentage Primary Need ASD,Percentage Primary Need OTH,Pupil,Building
URN,,,,,,,,,,,,,,,,,,,,
100000,Outstanding,2.952030,18.1,271.0,3628.0,142.0,London,Primary,0.738007,1.476015,0.000000,3.321033,11.439114,0.738007,0.738007,0.000000,3.321033,0.369004,"[100000, 102285, 102892, 101544, 102379, 10249...","[100000, 101110, 100010, 100245, 100125, 10002..."
100005,Outstanding,1.470588,44.3,107.5,1858.0,55.792578,London,Nursery,0.000000,0.000000,0.735294,0.000000,0.000000,0.000000,0.000000,0.735294,16.176471,0.735294,"[100005, 100767, 100315, 101105, 100554, 10149...","[100005, 100472, 100474, 100882, 101253, 10270..."
100006,Good,0.000000,68.4,49.0,1523.0,59.797111,London,Pupil referral unit,5.263158,0.000000,0.000000,52.631579,5.263158,0.000000,0.000000,0.000000,0.000000,0.000000,"[100006, 100391, 134919, 133749, 100994, 13526...","[100006, 100007, 102562, 102794, 108892, 13175..."
100007,Good,47.368421,100.0,19.0,1858.0,55.792578,London,Pupil referral unit,0.000000,0.000000,5.263158,68.421053,15.789474,0.000000,0.000000,0.000000,10.526316,0.000000,"[100007, 134597, 100391, 100006, 131584, 10099...","[100007, 102562, 102794, 108892, 131753, 13374..."
100008,Good,3.428571,55.1,350.0,2951.0,117.0,London,Primary,0.285714,8.285714,0.000000,1.428571,6.000000,0.000000,0.000000,0.000000,3.142857,1.142857,"[100008, 100408, 100341, 131740, 100403, 10041...","[100008, 131773, 102823, 100584, 102884, 10081..."
100009,Good,2.827763,43.4,389.0,3021.0,134.643496,London,Primary,3.084833,0.514139,0.000000,2.570694,7.969152,0.257069,0.257069,1.028278,1.799486,0.514139,"[100009, 102649, 100240, 100560, 102094, 10091...","[100009, 100996, 131595, 100896, 100907, 10080..."
100010,Good,3.550296,60.8,327.0,3415.0,142.0,London,Primary,1.479290,1.183432,0.000000,5.325444,6.804734,0.000000,0.000000,0.000000,5.325444,0.000000,"[100010, 131340, 131218, 101110, 101029, 13174...","[100010, 100245, 100783, 130358, 100000, 10042..."
100011,Good,1.015228,31.8,378.0,3817.0,72.965418,London,Primary,3.299492,0.000000,0.000000,3.807107,4.822335,0.000000,0.507614,0.000000,1.269036,0.253807,"[100011, 100229, 101815, 102399, 102635, 10190...","[100011, 102830, 101524, 136809, 102189, 10212..."
100013,Outstanding,3.000000,51.5,199.0,2552.0,7.0,London,Primary,0.500000,0.000000,0.000000,4.000000,18.000000,1.000000,1.000000,0.000000,0.500000,1.000000,"[100013, 100619, 100447, 100634, 100029, 10112...","[100013, 102921, 134307, 102568, 100732, 10186..."


Compute the rag calculation for the cost categories given in the [Benchmarking cost categories doc](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/DfEFinancialBenchmarking/_layouts/15/Doc.aspx?sourcedoc=%7B622FB0F9-7CB1-445A-8FFA-664F8857F036%7D&file=Benchmarking%20cost%20categories%20and%20sub-categories.docx&action=default&mobileredirect=true)

In [7]:
import warnings



In [8]:
rs = rag.compute_rag(raw_data, raw_comparators)
pd.DataFrame(rs)

Completed 100 RAGs in 4.46 secs
Completed 200 RAGs in 3.88 secs
Completed 300 RAGs in 3.87 secs
Completed 400 RAGs in 3.95 secs
Completed 500 RAGs in 3.94 secs
Completed 600 RAGs in 3.74 secs
Completed 700 RAGs in 3.80 secs
Completed 800 RAGs in 3.76 secs
Completed 900 RAGs in 3.80 secs
Completed 1000 RAGs in 3.79 secs
Completed 1100 RAGs in 4.30 secs
Completed 1200 RAGs in 3.79 secs
Completed 1300 RAGs in 3.82 secs
Completed 1400 RAGs in 3.77 secs
Completed 1500 RAGs in 3.84 secs
Completed 1600 RAGs in 3.81 secs
Completed 1700 RAGs in 3.83 secs
Completed 1800 RAGs in 3.74 secs
Completed 1900 RAGs in 3.79 secs
Completed 2000 RAGs in 3.90 secs
Completed 2100 RAGs in 3.78 secs
Completed 2200 RAGs in 3.79 secs
Completed 2300 RAGs in 3.80 secs
Completed 2400 RAGs in 3.76 secs
Completed 2500 RAGs in 3.79 secs
Completed 2600 RAGs in 3.77 secs
Completed 2700 RAGs in 3.82 secs
Completed 2800 RAGs in 3.79 secs
Completed 2900 RAGs in 3.74 secs
Completed 3000 RAGs in 3.78 secs
Completed 3100 RAGs

,Urn,Category,Sub category,Value,Mean,Diff Mean,Key,Percentage Diff,Percentile,Decile,Rag
0,100000,Teaching and Teaching support staff,Teaching staff,4219.955498,2632.192908,1587.762590,outstanding,37.625103,99,9,red
1,100000,Teaching and Teaching support staff,Supply teaching staff,45.340923,6.815345,38.525577,outstanding,84.968667,38,3,green
2,100000,Teaching and Teaching support staff,Education support staff,2329.561808,1213.818819,1115.742989,outstanding,47.894973,99,9,red
3,100000,Teaching and Teaching support staff,Agency supply teaching staff,266.307528,98.584710,167.722817,outstanding,62.980877,72,7,amber
4,100000,Teaching and Teaching support staff,Educational consultancy,179.885793,79.224911,100.660882,outstanding,55.958217,86,8,amber
...,...,...,...,...,...,...,...,...,...,...,...
450872,148841,Other costs,Special facilities,0.000000,5.327200,-5.327200,other,0.000000,0,0,green
450873,148841,Other costs,PFI charges,0.000000,0.000000,0.000000,other,0.000000,0,0,green
450874,148841,Other costs,Interest charges for loan and bank,0.000000,0.000000,0.000000,other,0.000000,0,0,green
450875,148841,Other costs,Direct revenue financing,0.000000,0.000000,0.000000,other,0.000000,0,0,green


In [7]:
%lprun -f rag.compute_category_rag list(compute_rag(raw_data, raw_comparators))